In [1]:
import cv2
import os
from google.colab.patches import cv2_imshow

In [3]:
class Cartoonizer: 
	def __init__(self): 
		pass

	def render(self, img_rgb): 
		img_rgb = cv2.imread(img_rgb) 
		img_rgb = cv2.resize(img_rgb, (1400,1400)) 
		numDownSamples = 2
		numBilateralFilters = 50  

		# downsample image using Gaussian pyramid 
		img_color = img_rgb 
		for _ in range(numDownSamples): 
			img_color = cv2.pyrDown(img_color) 

		# repeatedly apply small bilateral filter instead of applying one large filter 
		for _ in range(numBilateralFilters): 
			img_color = cv2.bilateralFilter(img_color, 15, 30, 20) 

		# upsample image to original size 
		for _ in range(numDownSamples): 
			img_color = cv2.pyrUp(img_color) 
 
		# convert to grayscale and apply median blur 
		img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY) 
		img_blur = cv2.medianBlur(img_gray, 3) 

		# detect and enhance edges 
		img_edge = cv2.adaptiveThreshold(img_blur, 255, 
										cv2.ADAPTIVE_THRESH_MEAN_C, 
										cv2.THRESH_BINARY, 3, 2) 

		# convert back to color so that it can be bit-ANDed with color image 
		(x,y,z) = img_color.shape 
		img_edge = cv2.resize(img_edge,(y,x)) 
		img_edge = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2RGB) 
		cv2.imwrite("edge.png",img_edge) 
		return cv2.bitwise_and(img_color, img_edge) 

tmp_canvas = Cartoonizer() 

file_name = "/content/dv.jpg"
res = tmp_canvas.render(file_name) 

cv2.imwrite("/content/Cartoon version.jpg", res) 
# cv2_imshow("Cartoon version") 
cv2.waitKey(0) 
cv2.destroyAllWindows() 
